## Init

In [1]:
import nltk
nltk.download("stopwords")
nltk.download('punkt')
import re
import pandas as pd
import os
from os.path import isfile, isdir, join
import numpy as np
import string
import nltk
import time
from keybert import KeyBERT
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForSequenceClassification

import random, pickle
from torch.nn import BCEWithLogitsLoss, BCELoss
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score, precision_recall_fscore_support
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import copy
from sklearn.utils import shuffle
import glob

import datetime
import json

from functools import partial

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer

from textacy import extract
import textacy
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DevNMor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DevNMor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
if torch.cuda.is_available():      
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

stop_words = stopwords.words('english')
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=False, output_hidden_states=True)
bert_model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

## Preparation

In [3]:
keyword_file = 'keyword.csv'
keywords = pd.read_csv(keyword_file)

data = pd.read_pickle("paragraph_match.pkl")
df1 = pd.read_csv("append_1.csv")
df2 = pd.read_csv("append_2.csv")

data = pd.concat([data, df1.iloc[:,1:], df2.iloc[:,1:]]).reset_index(drop=True)

## Function

In [6]:
def get_pretrained_wordvector(sentences, tokenizer, bert_model, max_length = 200):

    input_ids = []
    attention_masks = []
    char_to_tokens = []

    # Tokenize each sentence
    for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.

        tokenized_batch : BatchEncoding = tokenizer(sent, truncation=True,
                              add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                              max_length = max_length,           # Pad & truncate all sentences.
                              pad_to_max_length = True,
                              return_attention_mask = True,   # Construct attn. masks.
                              return_tensors = 'pt')

        # Add the encoded sentence to the list.    
        input_ids.append(tokenized_batch['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(tokenized_batch['attention_mask'])

        # align tokens with characters
        tokenized_text :Encoding  =tokenized_batch[0]
        tokens = tokenized_text.tokens

        alignment = {i: tokenized_text.char_to_token(i) for i in range(len(sent)) if tokenized_text.char_to_token(i) is not None}
        alignment = {i: (alignment[i], tokens[alignment[i]]) for i in alignment }
        char_to_tokens.append(alignment)



    #print(input_ids[0])
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    bert_model.eval()
    with torch.no_grad():
        outputs = bert_model(input_ids.to(device), attention_masks.to(device))   
        hidden_states = outputs[2]

    #print(len(hidden_states))
    # get the last four layers
    token_embeddings = torch.stack(hidden_states[-4:], dim=0) 
    #print(token_embeddings.size())

    # permute axis
    token_embeddings = token_embeddings.permute(1,2,0,3)
    #print(token_embeddings.size())

    # take the mean of the last 4 layers
    token_embeddings = token_embeddings.mean(axis=2)

    #print(token_embeddings.size())

    return token_embeddings, char_to_tokens


def get_keyword_emb(paragraph, seed_words, ngram_range, all_seed_words, sim_thresh = 0.5, top = 5):

    seed_words = list(seed_words)
    sentences = nltk.sent_tokenize(paragraph.lower())
    sent_emb, char_to_tokens = get_pretrained_wordvector(sentences, tokenizer, bert_model)

    en = textacy.load_spacy_lang("en_core_web_sm", disable=("parser",))

    # get ngram emb  # to do: remove stop words
    ngram_embs = []
    ngram_words = []
    seed_embs = []
    found_seed_words = []

    word_pairs = []

    for i,sent in enumerate(sentences):

        doc = textacy.make_spacy_doc(sent, lang=en)
        tokens = [t for t in doc]

        ngrams = [t for t in extract.basics.ngrams(doc, n = ngram_range)]
        #print(ngrams)

    # get emb of seed words
    for s in seed_words:
        if s in sent:
            start = sent.index(s.lower())
            end = start + len(s)
            s_token_ids = list(set([char_to_tokens[i][x][0] for x in range(start, end+1) if x in char_to_tokens[i]]))
            s_gram_emb = sent_emb[i][s_token_ids].mean(dim = 0)      
            seed_embs.append(s_gram_emb.unsqueeze(0))
            found_seed_words.append(s)


    # get emb of ngrams
    for ngram in ngrams:

      # if a seed word is a part of a ngram, or ngram is already in the seed word list, ignore the ngram
        if (len([w for w in seed_words if w in ngram.text]) == 0) and (ngram.text not in all_seed_words): 

            start = tokens[ngram.start].idx # start of the span
            end = tokens[ngram.end -1].idx + len(tokens[ngram.end -1 ]) # start of the span

            n_gram_token_ids = list(set([char_to_tokens[i][x][0] for x in range(start, end+1) if x in char_to_tokens[i]]))
            #print(n_gram_token_ids)

            n_gram_emb = sent_emb[i][n_gram_token_ids].mean(dim = 0)      

            ngram_embs.append(n_gram_emb.unsqueeze(0))
            ngram_words.append(ngram.text)

    #print(ngram_words)

    # calculate similarity 

    top_words = []
    top_words_emb = []

    if len(seed_embs) > 0  and len(ngram_embs)>0:
        seed_embs = F.normalize(torch.concat(seed_embs, axis = 0), dim = 1)
        ngram_embs = F.normalize(torch.concat(ngram_embs, axis = 0), dim = 1)

        sim = seed_embs.mm(ngram_embs.T).cpu().numpy()
        a, b = np.where(sim > sim_thresh)
        #print(a, b)
        #print(found_seed_words, seed_words,"\n", ngram_words, sim.shape)                  
        for x, y in zip(list(a), list(b)):
          #print(x, y)
            word_pairs.append([found_seed_words[x],ngram_words[y], sim[x,y] ])

        t1 = sim.argsort(axis = 1)[:, ::-1][:,0:top]
        t2 = list(b)

        tops = [(x, ngram_words[x]) for x in set(list(t1.reshape(-1))) if x in t2 ]

        if len(tops) > 0:
            top_ids, top_words = zip(*tops)
            top_ids = list(top_ids)
            top_words = list(top_words)
            top_words_emb = ngram_embs[top_ids]

        else:
            top_words_emb = None
            seed_embs = None


    return top_words, top_words_emb, seed_embs, word_pairs
    

## Run

In [7]:
data_0 = data.loc[600000:,:].reset_index(drop=True)

In [10]:
ngram_range=(2,3)
# data["new_words"] = None
new_words = []
new_pairs = []
start = time.time()
start_idx = 0
# res = pd.DataFrame(columns=['cik', 'file', 'paragraph', 'keyword', 'new_words'])
new_words = new_words[0:start_idx]
new_pairs= new_pairs[0:start_idx]

for i, row in data_0.iterrows():
    try:
        paragraph = row["paragraph"]
        seed_words = row["keyword"]

        top_words, _, _, word_pairs = get_keyword_emb(paragraph, seed_words, 
                                                    all_seed_words = keywords["keywords"],
                                                  ngram_range = ngram_range,
                                                  sim_thresh = 0.5,top = 5)
        # res.loc[i] = [row.cik, row.file, row.paragraph, row.keyword, top_words]
        new_words.append(top_words)
        new_pairs += word_pairs

        if i%1000 == 0 :
            print(f"complete {i} row of {len(data_0)} in {(time.time()-start):.2f}")
    except:
        pd.DataFrame(new_words).to_csv('temp_save_3.1.csv')
        df = pd.DataFrame(new_pairs, columns =['w1','w2','sim'])
        df = df.groupby(["w1","w2"]).mean().reset_index()
        df.to_csv('new_words_3.1.csv', header = True, index = False)
        print(f'problem happens in index {i}')
        break

    # data["new_words"] = new_words

complete 0 row of 330293 in 4.81
complete 1000 row of 330293 in 35.88
complete 2000 row of 330293 in 61.07
complete 3000 row of 330293 in 87.92
complete 4000 row of 330293 in 117.44
complete 5000 row of 330293 in 147.81
complete 6000 row of 330293 in 176.62
complete 7000 row of 330293 in 206.21
complete 8000 row of 330293 in 236.94
complete 9000 row of 330293 in 267.83
complete 10000 row of 330293 in 298.75
complete 11000 row of 330293 in 329.96
complete 12000 row of 330293 in 359.48
complete 13000 row of 330293 in 386.82
complete 14000 row of 330293 in 416.58
complete 15000 row of 330293 in 446.44
complete 16000 row of 330293 in 478.30
complete 17000 row of 330293 in 509.96
complete 18000 row of 330293 in 539.49
complete 19000 row of 330293 in 568.73
complete 20000 row of 330293 in 596.20
complete 21000 row of 330293 in 628.82
complete 22000 row of 330293 in 661.28
complete 23000 row of 330293 in 696.28
complete 24000 row of 330293 in 728.76
complete 25000 row of 330293 in 762.74
comp

complete 204000 row of 330293 in 8875.19
complete 205000 row of 330293 in 8906.44
complete 206000 row of 330293 in 8936.00
complete 207000 row of 330293 in 8967.65
complete 208000 row of 330293 in 8999.07
complete 209000 row of 330293 in 9030.26
complete 210000 row of 330293 in 9061.03
complete 211000 row of 330293 in 9093.34
complete 212000 row of 330293 in 9123.73
complete 213000 row of 330293 in 9152.80
complete 214000 row of 330293 in 9183.10
complete 215000 row of 330293 in 9214.66
complete 216000 row of 330293 in 9244.40
complete 217000 row of 330293 in 9275.86
complete 218000 row of 330293 in 9307.26
complete 219000 row of 330293 in 9339.19
complete 220000 row of 330293 in 9369.97
complete 221000 row of 330293 in 9399.49
complete 222000 row of 330293 in 9431.18
complete 223000 row of 330293 in 9462.32
complete 224000 row of 330293 in 9493.29
complete 225000 row of 330293 in 9525.31
complete 226000 row of 330293 in 9557.83
complete 227000 row of 330293 in 9590.44
complete 228000 

In [11]:
pd.DataFrame(new_words).to_csv('temp_save_3.csv')
df = pd.DataFrame(new_pairs, columns =['w1','w2','sim'])
df = df.groupby(["w1","w2"]).mean().reset_index()
df.to_csv('new_words_3.csv', header = True, index = False)